### Analyze hub genes and high-weight edges.

#### Hub gene analysis

##### Obtain the gene connectivity matrix (for each cell type, there are five degree matrices, one for each fold)

In [ ]:
! python ../get_degree_matrix_train.py -expr ../../data/pre_data/scRNAseq_datasets/AMB.npz \
    -outdir ../../result/ \
    -ca 0.01 -hvgs 2000

##### Save the hub genes for each cell type for the five folds (100 genes for each fold).

In [ ]:
import pandas as pd
import numpy as np
import os
seq_dict = np.load('../../result/datasets/AMB/seq_dict.npz', allow_pickle=True) 
label = seq_dict['label']

matrix_dict = np.load('../../result/datasets/AMB/degree_matrix_train_AMB_a0.01_hvgs2000.npz', allow_pickle=True)

print("Keys in loaded matrix_dict:", matrix_dict.files)
str_labels = matrix_dict['str_labels']
print("cell-type: ", str_labels)

print("Length of str_labels:", len(str_labels))

for k in range(len(str_labels)):
    degree_matrix_key = f'{k}' 
    degree_matrix_dict = matrix_dict[degree_matrix_key].item()  
    print(degree_matrix_dict.keys())
    fold_top_indices_df = pd.DataFrame()

    for i in range(5):
        fold = i + 1
        cur_fold_degree_matrix = degree_matrix_dict[f'CV_{fold}']
        mean_degree = np.mean(cur_fold_degree_matrix, axis=1)
        top_100_indices = np.argsort(mean_degree)[-100:][::-1] 
        fold_top_indices_df[f'Fold_{fold}'] = top_100_indices

    cell_type_name = str_labels[k]
    os.makedirs("data/AMB/Gene_degree",exist_ok=True)
    tsv_filename = f'data/AMB/Gene_degree/{cell_type_name}_top100_indices.tsv'
    fold_top_indices_df.to_csv(tsv_filename, sep='\t', index=False)

    print(f"Saved top 100 indices for cell type '{cell_type_name}' to {tsv_filename}")

##### Obtain the union of hub genes for each cell type to get the charactersit gene set for each cell type.

In [ ]:
import pandas as pd
import os

def read_and_union_tsv(cell_type):

    file_path = f'data/AMB/Gene_degree/{cell_type}_top100_indices.tsv'
    
    try:
        data = pd.read_csv(file_path, sep='\t')
        union_set = set()
        for col in data.columns:
            union_set.update(data[col].dropna().astype(int))
        
        return list(union_set)
    except FileNotFoundError:
        print(f"Warning: File not found for cell type {cell_type}")
        return []
    except Exception as e:
        print(f"Error processing {cell_type}: {str(e)}")
        return []

def process_cell_types(str_labels):

    result_dict = {}

    for cell_type in str_labels:
        union_genes = read_and_union_tsv(cell_type)
        result_dict[cell_type] = pd.Series(union_genes, dtype='Int64')
        print(f"{cell_type}: {len(union_genes)} genes")
    
    result_df = pd.DataFrame(result_dict)
    
    return result_df

seq_dict = np.load('../../result/datasets/AMB/seq_dict.npz', allow_pickle=True) 
str_labels = seq_dict['str_labels']

result_df = process_cell_types(str_labels)

output_path = 'data/AMB/AMB_character_gene_set_indices.tsv'
result_df.to_csv(output_path, sep='\t', index=False)
print(f"\nResults saved to: {output_path}")


##### Save the gene symbols corresponding to the 2000 HVGs (Highly Variable Genes).

In [ ]:
seq_dict = np.load('../../result/datasets/AMB/seq_dict.npz', allow_pickle=True) 
genes = seq_dict['gene_symbol']

all_filtered_genes_file = '../../result/datasets/AMB/AMB_filtered_hvgs2000.npy'

all_filtered_genes_array = np.load(all_filtered_genes_file, allow_pickle=True)
filtered_genes_index = all_filtered_genes_array[0]
filtered_genes_index = filtered_genes_index.astype(int)
print(filtered_genes_index.shape)

filtered_genes = genes[filtered_genes_index]

print(filtered_genes)
print(filtered_genes.shape)
df = pd.DataFrame(filtered_genes, columns=['gene_symbol'])
df.to_csv('data/AMB/AMB_gene_symbol_hvgs2000.tsv',  sep='\t', index=False, header=True)

##### Based on the gene symbols obtained above, convert the indices in the character gene set to the corresponding gene symbols

In [ ]:
import pandas as pd

file_path = 'data/AMB/AMB_character_gene_set_indices.tsv'
gene_idx_df = pd.read_csv(file_path, sep='\t')

gene_symbol_file = 'data/AMB/AMB_gene_symbol_hvgs2000.tsv'
gene_symbol = pd.read_csv(gene_symbol_file, sep='\t')
gene_symbol_dict = {i: gene_symbol.iloc[i, 0] for i in range(len(gene_symbol))}

def map_to_gene_symbol(value):
    if pd.isna(value):
        return None
    return gene_symbol_dict.get(int(value), None)

gene_idx_df = gene_idx_df.applymap(map_to_gene_symbol)

output_file_path = 'data/AMB/AMB_character_gene_set.tsv'
gene_idx_df.to_csv(output_file_path, sep='\t', index=False)

print(f"Save to {output_file_path}")


#### hub gene Plot: R

In [ ]:
Rscript Figure-AMB-gene.R

#### Analysis of highly correlated gene pairs.

##### Obtain the top 100 high-weight edges (gene pairs) for each cell type

In [ ]:
import numpy as np
import os
import torch
from scipy import sparse
import heapq
from collections import defaultdict

def find_top_edges(matrices, top_k=100):

    total_matrices = len(matrices)  
    edge_weights = defaultdict(float)
    
    for matrix in matrices:
        rows, cols = matrix.nonzero()
        values = matrix.data

        for i in range(len(rows)):
            node1 = int(min(rows[i], cols[i]))
            node2 = int(max(rows[i], cols[i]))
            if node1 != node2:  
                edge_weights[(node1, node2)] += float(values[i])
    edge_avg_weights = []
    for (node1, node2), weight_sum in edge_weights.items():
        avg_weight = float(weight_sum) / float(total_matrices)
        edge_avg_weights.append((int(node1), int(node2), float(avg_weight)))

    return heapq.nlargest(top_k, edge_avg_weights, key=lambda x: x[2])


def load_and_process_matrices(cell_test_folder, cur_label_idxs):

    matrices = []

    for idx in cur_label_idxs:
        data = torch.load(os.path.join(cell_test_folder, f'cell_{idx}.pt'))
        edge_index = data.edge_index
        edge_weight = data.edge_weight
        
        num_nodes = data.x.shape[0] 
        edges = edge_index.cpu().numpy()
        weights = edge_weight.cpu().numpy()

        sparse_mat = sparse.csr_matrix(
            (weights, (edges[0], edges[1])),
            shape=(num_nodes, num_nodes)
        )           
      
        matrices.append(sparse_mat)     
    
    return matrices
    

def save_cell_type_edges(cell_type, fold_edges, save_folder, base_filename):

    save_dir = os.path.join(save_folder, f"{base_filename}_edges")
    os.makedirs(save_dir, exist_ok=True)
    
    filename = os.path.join(save_dir, f"{cell_type}_top_edges.npz")
    
    save_dict = {
        'cell_type': cell_type,
        'base_filename': base_filename
    }
    
    for fold_idx, edges in fold_edges.items():
        edges_array = np.array(edges, dtype=[
            ('node1', 'int32'), 
            ('node2', 'int32'), 
            ('weight', 'float32')
        ])
        save_dict[f'fold_{fold_idx}_edges'] = edges_array        
    
    np.savez(filename, **save_dict)


In [ ]:
seq_dict = np.load('../../result/datasets/AMB/seq_dict.npz', allow_pickle=True) 
label = seq_dict['label'] 
str_labels = seq_dict['str_labels']
save_folder = 'data/AMB/Edge_weight'

cell_type_edges = {}

for cur_label, cell_type in enumerate(str_labels):
    print("cur_label: ", cur_label)
    print("cell_type: ", cell_type)

    fold_edges = {}
    
    for k in range(5):
        k_fold = k + 1
        train_index = seq_dict[f'train_index_{k_fold}']
        label_train = label[train_index]
        cur_label_idxs = np.where(label_train == cur_label)[0].tolist()


        cell_train_folder = os.path.join(
            "../../result/datasets/AMB/wcsn_a0.01_hvgs2000", 
            f"train_f{k_fold}", 
            'processed'
        )
        
        cur_mat = load_and_process_matrices(cell_train_folder, cur_label_idxs)
        cur_top_edges = find_top_edges(cur_mat)

        fold_edges[k_fold] = cur_top_edges

    save_cell_type_edges(cell_type, fold_edges, save_folder,  base_filename="AMB")

In [ ]:
import numpy as np
import pandas as pd
import os
import glob

def create_edge_strings(edges):
    return [f"{int(edge[0])}-{int(edge[1])}" for edge in edges]

def process_npz_file(file_path):
    cell_type = os.path.basename(file_path).replace('_top_edges.npz', '')
    
    output_dir = os.path.join('data/AMB/Edge_weight', f'{cell_type}_top_edges')
    
    os.makedirs(output_dir, exist_ok=True)
    try:
        data = np.load(file_path, allow_pickle=True)

        for i in range(1, 6):  
            fold_key = f'fold_{i}_edges'
            if fold_key in data:
                edges = data[fold_key]
                edge_strings = create_edge_strings(edges)
                
                output_file = os.path.join(output_dir, f'fold_{i}_edges.csv')
                pd.DataFrame({'edges': edge_strings}).to_csv(output_file, index=False)
                
        print(f"Successfully processed {cell_type}")
        
    except Exception as e:
        print(f"Error processing {cell_type}: {str(e)}")

def process_all_cell_types(base_dir):

    npz_files = glob.glob(os.path.join(base_dir, '*_top_edges.npz'))
    
    if not npz_files:
        print(f"No npz files found in {base_dir}")
        return
    
    print(f"Found {len(npz_files)} files to process")
    
    for file_path in npz_files:
        process_npz_file(file_path)
    
    print("Processing complete!")

base_dir = 'data/AMB/Edge_weight/AMB_edges'
process_all_cell_types(base_dir)

##### Obtain the union of highly correlated gene pairs for each cell type to get the characterist edge set for each cell type.

In [ ]:
import numpy as np
import pandas as pd
import os

def get_union_edges(npz_file):
   data = np.load(npz_file, allow_pickle=True)
   all_edges = set()
   
   for i in range(1, 6):
       fold_key = f'fold_{i}_edges'
       if fold_key in data:
           edges = data[fold_key]
           edge_strings = [f"{int(edge[0])}-{int(edge[1])}" for edge in edges]
           all_edges.update(edge_strings)
   print(list(all_edges))
   return sorted(list(all_edges)) 


seq_dict = np.load('../../result/datasets/AMB/seq_dict.npz', allow_pickle=True) 
str_labels = seq_dict['str_labels']

cell_types = str_labels.tolist()  
print("cell types: ", cell_types)

edges_dict = {}

for cell_type in cell_types:
   npz_file = f'data/AMB/Edge_weight/AMB_edges/{cell_type}_top_edges.npz' 
   if os.path.exists(npz_file):
        edges_dict[cell_type] = get_union_edges(npz_file)

max_length = max(len(edges) for edges in edges_dict.values())

for cell_type in edges_dict:
   if len(edges_dict[cell_type]) < max_length:
       edges_dict[cell_type].extend([''] * (max_length - len(edges_dict[cell_type])))

df = pd.DataFrame(edges_dict)
print(df.columns)

save_path = 'data/AMB/AMB_character_edge_set_indices.tsv'
df.to_csv(save_path, sep='\t', index=False)

print(f"Save successfully: {save_path}")

print("\n The number of different cell types")

for cell_type in cell_types:
   edge_count = len([x for x in edges_dict[cell_type] if x != ''])
   print(f"{cell_type}: {edge_count}")

##### Convert the indices in the character edge set file obtained above to the corresponding gene pair gene symbols and save them.

In [ ]:
import pandas as pd

gene_symbol_file = 'data/AMB/AMB_gene_symbol_hvgs2000.tsv'
gene_symbol = pd.read_csv(gene_symbol_file, sep='\t')

gene_symbol_dict = {i: gene_symbol.iloc[i, 0] for i in range(len(gene_symbol))}

file_path = 'data/AMB/AMB_character_edge_set_indices.tsv'
df = pd.read_csv(file_path, sep='\t')

def convert_edges_to_gene_symbols(edge_str):
    if pd.isna(edge_str):
        return None
    gene1, gene2 = edge_str.split('-')
    gene1_symbol = gene_symbol_dict.get(int(gene1), 'Unknown')
    gene2_symbol = gene_symbol_dict.get(int(gene2), 'Unknown')
    return f"{gene1_symbol}-{gene2_symbol}"

for column in df.columns:
    df[column] = df[column].apply(convert_edges_to_gene_symbols)

save_path = 'data/AMB/AMB_character_edge_set.tsv'
df.to_csv(save_path, sep='\t', index=False)

print(f"Save to: {save_path}")

print("\n The number of different cell types")
for column in df.columns:
    edge_count = df[column].apply(lambda x: x != '').sum()
    print(f"{column}: {edge_count}")


#### Plot highly correlated gene pairs: R

In [ ]:
Rscript Figure-AMB-edge.R